# Эксперименты с Random Forest и Gradient Boosting

**Датасет**: House Sales in King County, USA

**Цель**: Исследовать поведение собственных реализаций Random Forest и Gradient Boosting на реальных данных.

## Содержание
1. Загрузка и предобработка данных
2. Эксперименты с Random Forest
3. Эксперименты с Gradient Boosting
4. Сравнение результатов

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

from ensembles.random_forest import RandomForestMSE
from ensembles.boosting import GradientBoostingMSE

plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")
%matplotlib inline

In [ ]:
## 1. Загрузка и предобработка данных

# Загрузка данных
DATA_PATH = Path("../data/kc_house_data.csv")
df = pd.read_csv(DATA_PATH)

print(f"Размер датасета: {df.shape}")
print(f"\nПервые 5 строк:")
df.head()

In [ ]:
# Информация о данных
print("Информация о столбцах:")
print(df.info())
print("\n" + "="*50)
print("\nСтатистика:")
df.describe()

In [ ]:
### Предобработка данных

# 1. Извлекаем признаки из даты
df["date"] = pd.to_datetime(df["date"])
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"] = df["date"].dt.day

# 2. Удаляем ненужные столбцы
df = df.drop(columns=["date", "id"])

# 3. Разделяем на признаки и целевую переменную
target = df.pop("price")
X = df.copy()

# 4. Обрабатываем категориальные признаки (zipcode)
# Используем one-hot encoding для zipcode
X = pd.get_dummies(X, columns=["zipcode"], prefix="zip")

print(f"Размер признаков после предобработки: {X.shape}")
print(f"Целевая переменная (price): min={target.min():.0f}, max={target.max():.0f}, mean={target.mean():.0f}")

In [ ]:
### Разделение данных на train/validation/test

# Сначала отделяем тестовую выборку (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, target, test_size=0.2, random_state=42, shuffle=True
)

# Затем разделяем оставшиеся данные на train (64%) и validation (16%)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, shuffle=True
)

# Преобразуем в numpy массивы
X_train_np = X_train.values.astype(np.float64)
X_val_np = X_val.values.astype(np.float64)
X_test_np = X_test.values.astype(np.float64)
y_train_np = y_train.values.astype(np.float64)
y_val_np = y_val.values.astype(np.float64)
y_test_np = y_test.values.astype(np.float64)

print("Размеры выборок:")
print(f"  Train: {X_train_np.shape[0]} объектов")
print(f"  Validation: {X_val_np.shape[0]} объектов")
print(f"  Test: {X_test_np.shape[0]} объектов")
print(f"  Количество признаков: {X_train_np.shape[1]}")

## 2. Эксперименты с Random Forest

### 2.1. Зависимость от количества деревьев

In [ ]:
# Исследуем влияние количества деревьев
n_estimators_list = [5, 10, 20, 30, 50, 75, 100]

results_rf_n_trees = []

for n_trees in n_estimators_list:
    print(f"Обучение RF с {n_trees} деревьями...")
    
    rf = RandomForestMSE(
        n_estimators=n_trees,
        tree_params={"max_depth": 10, "random_state": 42}
    )
    
    start_time = time.time()
    rf.fit(X_train_np, y_train_np)
    train_time = time.time() - start_time
    
    # Предсказания
    y_pred_val = rf.predict(X_val_np)
    y_pred_test = rf.predict(X_test_np)
    
    # RMSE
    rmse_val = np.sqrt(mean_squared_error(y_val_np, y_pred_val))
    rmse_test = np.sqrt(mean_squared_error(y_test_np, y_pred_test))
    
    results_rf_n_trees.append({
        'n_estimators': n_trees,
        'rmse_val': rmse_val,
        'rmse_test': rmse_test,
        'time': train_time
    })
    
    print(f"  RMSE (val): {rmse_val:,.0f}, RMSE (test): {rmse_test:,.0f}, Time: {train_time:.2f}s\n")

df_rf_n_trees = pd.DataFrame(results_rf_n_trees)
df_rf_n_trees

In [ ]:
# Визуализация зависимости от количества деревьев
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# RMSE
axes[0].plot(df_rf_n_trees['n_estimators'], df_rf_n_trees['rmse_val'], 'o-', label='Validation', linewidth=2)
axes[0].plot(df_rf_n_trees['n_estimators'], df_rf_n_trees['rmse_test'], 's-', label='Test', linewidth=2)
axes[0].set_xlabel('Количество деревьев', fontsize=12)
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('Random Forest: RMSE vs Количество деревьев', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Время обучения
axes[1].plot(df_rf_n_trees['n_estimators'], df_rf_n_trees['time'], 'o-', color='green', linewidth=2)
axes[1].set_xlabel('Количество деревьев', fontsize=12)
axes[1].set_ylabel('Время обучения (сек)', fontsize=12)
axes[1].set_title('Random Forest: Время обучения vs Количество деревьев', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.2. Зависимость от max_features (размерность подвыборки признаков)

In [ ]:
# Исследуем влияние max_features
n_features = X_train_np.shape[1]
max_features_list = [int(np.sqrt(n_features)), n_features // 4, n_features // 2, n_features]

results_rf_max_features = []

for max_feat in max_features_list:
    print(f"Обучение RF с max_features={max_feat}...")
    
    rf = RandomForestMSE(
        n_estimators=50,
        tree_params={"max_depth": 10, "max_features": max_feat, "random_state": 42}
    )
    
    start_time = time.time()
    rf.fit(X_train_np, y_train_np)
    train_time = time.time() - start_time
    
    y_pred_val = rf.predict(X_val_np)
    y_pred_test = rf.predict(X_test_np)
    
    rmse_val = np.sqrt(mean_squared_error(y_val_np, y_pred_val))
    rmse_test = np.sqrt(mean_squared_error(y_test_np, y_pred_test))
    
    results_rf_max_features.append({
        'max_features': max_feat,
        'max_features_ratio': max_feat / n_features,
        'rmse_val': rmse_val,
        'rmse_test': rmse_test,
        'time': train_time
    })
    
    print(f"  RMSE (val): {rmse_val:,.0f}, RMSE (test): {rmse_test:,.0f}, Time: {train_time:.2f}s\n")

df_rf_max_features = pd.DataFrame(results_rf_max_features)
df_rf_max_features

### 2.3. Зависимость от максимальной глубины дерева

In [ ]:
# Исследуем влияние глубины дерева (включая None - неограниченная глубина)
max_depth_list = [3, 5, 10, 15, 20, None]

results_rf_max_depth = []

for max_d in max_depth_list:
    depth_str = str(max_d) if max_d is not None else "Unlimited"
    print(f"Обучение RF с max_depth={depth_str}...")
    
    rf = RandomForestMSE(
        n_estimators=30,
        tree_params={"max_depth": max_d, "random_state": 42}
    )
    
    start_time = time.time()
    rf.fit(X_train_np, y_train_np)
    train_time = time.time() - start_time
    
    y_pred_val = rf.predict(X_val_np)
    y_pred_test = rf.predict(X_test_np)
    
    rmse_val = np.sqrt(mean_squared_error(y_val_np, y_pred_val))
    rmse_test = np.sqrt(mean_squared_error(y_test_np, y_pred_test))
    
    results_rf_max_depth.append({
        'max_depth': depth_str,
        'max_depth_num': max_d if max_d is not None else 999,
        'rmse_val': rmse_val,
        'rmse_test': rmse_test,
        'time': train_time
    })
    
    print(f"  RMSE (val): {rmse_val:,.0f}, RMSE (test): {rmse_test:,.0f}, Time: {train_time:.2f}s\n")

df_rf_max_depth = pd.DataFrame(results_rf_max_depth)
df_rf_max_depth

In [ ]:
# Визуализация для max_depth
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# RMSE
x_labels = df_rf_max_depth['max_depth'].values
x_pos = np.arange(len(x_labels))
axes[0].bar(x_pos - 0.2, df_rf_max_depth['rmse_val'], 0.4, label='Validation', alpha=0.8)
axes[0].bar(x_pos + 0.2, df_rf_max_depth['rmse_test'], 0.4, label='Test', alpha=0.8)
axes[0].set_xticks(x_pos)
axes[0].set_xticklabels(x_labels, rotation=0)
axes[0].set_xlabel('Максимальная глубина', fontsize=12)
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('Random Forest: RMSE vs Максимальная глубина', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

# Время
axes[1].bar(x_pos, df_rf_max_depth['time'], color='green', alpha=0.8)
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(x_labels, rotation=0)
axes[1].set_xlabel('Максимальная глубина', fontsize=12)
axes[1].set_ylabel('Время обучения (сек)', fontsize=12)
axes[1].set_title('Random Forest: Время обучения vs Максимальная глубина', fontsize=14)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 3. Эксперименты с Gradient Boosting

### 3.1. Зависимость от количества деревьев

In [ ]:
# Исследуем влияние количества деревьев для GB
n_estimators_gb_list = [10, 20, 30, 50, 75, 100, 150]

results_gb_n_trees = []

for n_trees in n_estimators_gb_list:
    print(f"Обучение GB с {n_trees} деревьями...")
    
    gb = GradientBoostingMSE(
        n_estimators=n_trees,
        tree_params={"max_depth": 3, "random_state": 42},
        learning_rate=0.1
    )
    
    start_time = time.time()
    gb.fit(X_train_np, y_train_np)
    train_time = time.time() - start_time
    
    y_pred_val = gb.predict(X_val_np)
    y_pred_test = gb.predict(X_test_np)
    
    rmse_val = np.sqrt(mean_squared_error(y_val_np, y_pred_val))
    rmse_test = np.sqrt(mean_squared_error(y_test_np, y_pred_test))
    
    results_gb_n_trees.append({
        'n_estimators': n_trees,
        'rmse_val': rmse_val,
        'rmse_test': rmse_test,
        'time': train_time
    })
    
    print(f"  RMSE (val): {rmse_val:,.0f}, RMSE (test): {rmse_test:,.0f}, Time: {train_time:.2f}s\n")

df_gb_n_trees = pd.DataFrame(results_gb_n_trees)
df_gb_n_trees

In [ ]:
# Визуализация GB: количество деревьев
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(df_gb_n_trees['n_estimators'], df_gb_n_trees['rmse_val'], 'o-', label='Validation', linewidth=2)
axes[0].plot(df_gb_n_trees['n_estimators'], df_gb_n_trees['rmse_test'], 's-', label='Test', linewidth=2)
axes[0].set_xlabel('Количество деревьев', fontsize=12)
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('Gradient Boosting: RMSE vs Количество деревьев', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

axes[1].plot(df_gb_n_trees['n_estimators'], df_gb_n_trees['time'], 'o-', color='green', linewidth=2)
axes[1].set_xlabel('Количество деревьев', fontsize=12)
axes[1].set_ylabel('Время обучения (сек)', fontsize=12)
axes[1].set_title('Gradient Boosting: Время обучения vs Количество деревьев', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.2. Зависимость от max_features

In [ ]:
# Влияние max_features для GB
results_gb_max_features = []

for max_feat in max_features_list:
    print(f"Обучение GB с max_features={max_feat}...")
    
    gb = GradientBoostingMSE(
        n_estimators=50,
        tree_params={"max_depth": 3, "max_features": max_feat, "random_state": 42},
        learning_rate=0.1
    )
    
    start_time = time.time()
    gb.fit(X_train_np, y_train_np)
    train_time = time.time() - start_time
    
    y_pred_val = gb.predict(X_val_np)
    y_pred_test = gb.predict(X_test_np)
    
    rmse_val = np.sqrt(mean_squared_error(y_val_np, y_pred_val))
    rmse_test = np.sqrt(mean_squared_error(y_test_np, y_pred_test))
    
    results_gb_max_features.append({
        'max_features': max_feat,
        'max_features_ratio': max_feat / n_features,
        'rmse_val': rmse_val,
        'rmse_test': rmse_test,
        'time': train_time
    })
    
    print(f"  RMSE (val): {rmse_val:,.0f}, RMSE (test): {rmse_test:,.0f}, Time: {train_time:.2f}s\n")

df_gb_max_features = pd.DataFrame(results_gb_max_features)
df_gb_max_features

### 3.3. Зависимость от максимальной глубины

In [ ]:
# Влияние глубины для GB (включая неограниченную)
max_depth_gb_list = [2, 3, 5, 7, 10, None]

results_gb_max_depth = []

for max_d in max_depth_gb_list:
    depth_str = str(max_d) if max_d is not None else "Unlimited"
    print(f"Обучение GB с max_depth={depth_str}...")
    
    gb = GradientBoostingMSE(
        n_estimators=50,
        tree_params={"max_depth": max_d, "random_state": 42},
        learning_rate=0.1
    )
    
    start_time = time.time()
    gb.fit(X_train_np, y_train_np)
    train_time = time.time() - start_time
    
    y_pred_val = gb.predict(X_val_np)
    y_pred_test = gb.predict(X_test_np)
    
    rmse_val = np.sqrt(mean_squared_error(y_val_np, y_pred_val))
    rmse_test = np.sqrt(mean_squared_error(y_test_np, y_pred_test))
    
    results_gb_max_depth.append({
        'max_depth': depth_str,
        'max_depth_num': max_d if max_d is not None else 999,
        'rmse_val': rmse_val,
        'rmse_test': rmse_test,
        'time': train_time
    })
    
    print(f"  RMSE (val): {rmse_val:,.0f}, RMSE (test): {rmse_test:,.0f}, Time: {train_time:.2f}s\n")

df_gb_max_depth = pd.DataFrame(results_gb_max_depth)
df_gb_max_depth

### 3.4. Зависимость от learning_rate

In [ ]:
# Влияние learning_rate для GB
learning_rate_list = [0.01, 0.05, 0.1, 0.2, 0.3, 0.5]

results_gb_lr = []

for lr in learning_rate_list:
    print(f"Обучение GB с learning_rate={lr}...")
    
    gb = GradientBoostingMSE(
        n_estimators=50,
        tree_params={"max_depth": 3, "random_state": 42},
        learning_rate=lr
    )
    
    start_time = time.time()
    gb.fit(X_train_np, y_train_np)
    train_time = time.time() - start_time
    
    y_pred_val = gb.predict(X_val_np)
    y_pred_test = gb.predict(X_test_np)
    
    rmse_val = np.sqrt(mean_squared_error(y_val_np, y_pred_val))
    rmse_test = np.sqrt(mean_squared_error(y_test_np, y_pred_test))
    
    results_gb_lr.append({
        'learning_rate': lr,
        'rmse_val': rmse_val,
        'rmse_test': rmse_test,
        'time': train_time
    })
    
    print(f"  RMSE (val): {rmse_val:,.0f}, RMSE (test): {rmse_test:,.0f}, Time: {train_time:.2f}s\n")

df_gb_lr = pd.DataFrame(results_gb_lr)
df_gb_lr

In [ ]:
# Визуализация для learning_rate
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(df_gb_lr['learning_rate'], df_gb_lr['rmse_val'], 'o-', label='Validation', linewidth=2)
axes[0].plot(df_gb_lr['learning_rate'], df_gb_lr['rmse_test'], 's-', label='Test', linewidth=2)
axes[0].set_xlabel('Learning Rate', fontsize=12)
axes[0].set_ylabel('RMSE', fontsize=12)
axes[0].set_title('Gradient Boosting: RMSE vs Learning Rate', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

axes[1].plot(df_gb_lr['learning_rate'], df_gb_lr['time'], 'o-', color='green', linewidth=2)
axes[1].set_xlabel('Learning Rate', fontsize=12)
axes[1].set_ylabel('Время обучения (сек)', fontsize=12)
axes[1].set_title('Gradient Boosting: Время обучения vs Learning Rate', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Итоговое сравнение и выводы

In [ ]:
# Обучим лучшие модели на основе экспериментов

# Random Forest: лучшая конфигурация
rf_best = RandomForestMSE(
    n_estimators=75,
    tree_params={"max_depth": 15, "random_state": 42}
)

start = time.time()
rf_best.fit(X_train_np, y_train_np)
rf_train_time = time.time() - start

y_pred_rf = rf_best.predict(X_test_np)
rmse_rf = np.sqrt(mean_squared_error(y_test_np, y_pred_rf))

# Gradient Boosting: лучшая конфигурация
gb_best = GradientBoostingMSE(
    n_estimators=100,
    tree_params={"max_depth": 5, "random_state": 42},
    learning_rate=0.1
)

start = time.time()
gb_best.fit(X_train_np, y_train_np)
gb_train_time = time.time() - start

y_pred_gb = gb_best.predict(X_test_np)
rmse_gb = np.sqrt(mean_squared_error(y_test_np, y_pred_gb))

# Сравнительная таблица
comparison = pd.DataFrame({
    'Модель': ['Random Forest', 'Gradient Boosting'],
    'RMSE (test)': [rmse_rf, rmse_gb],
    'Время обучения (сек)': [rf_train_time, gb_train_time]
})

print("="*60)
print("ИТОГОВОЕ СРАВНЕНИЕ МОДЕЛЕЙ")
print("="*60)
print(comparison.to_string(index=False))
print("="*60)

In [ ]:
# Финальная визуализация: сравнение предсказаний
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Random Forest
axes[0].scatter(y_test_np, y_pred_rf, alpha=0.5, s=20)
axes[0].plot([y_test_np.min(), y_test_np.max()], [y_test_np.min(), y_test_np.max()], 
             'r--', lw=2, label='Идеальное предсказание')
axes[0].set_xlabel('Истинные значения', fontsize=12)
axes[0].set_ylabel('Предсказанные значения', fontsize=12)
axes[0].set_title(f'Random Forest\nRMSE = {rmse_rf:,.0f}', fontsize=14)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Gradient Boosting
axes[1].scatter(y_test_np, y_pred_gb, alpha=0.5, s=20, color='orange')
axes[1].plot([y_test_np.min(), y_test_np.max()], [y_test_np.min(), y_test_np.max()], 
             'r--', lw=2, label='Идеальное предсказание')
axes[1].set_xlabel('Истинные значения', fontsize=12)
axes[1].set_ylabel('Предсказанные значения', fontsize=12)
axes[1].set_title(f'Gradient Boosting\nRMSE = {rmse_gb:,.0f}', fontsize=14)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
### Выводы

print("""
ВЫВОДЫ ИЗ ЭКСПЕРИМЕНТОВ:

1. RANDOM FOREST:
   - Увеличение количества деревьев улучшает качество, но рост замедляется после 50-75 деревьев
   - Оптимальная глубина деревьев: 10-15 (глубже - переобучение, мельче - недообучение)
   - Время обучения растёт линейно с количеством деревьев
   - Неограниченная глубина приводит к переобучению и увеличению времени обучения
   
2. GRADIENT BOOSTING:
   - Более чувствителен к количеству деревьев - качество продолжает улучшаться дольше
   - Оптимальная глубина базовых деревьев: 3-5 (мелкие деревья работают лучше)
   - Learning rate 0.1 обеспечивает хороший баланс между качеством и скоростью
   - Слишком большой learning_rate (>0.3) ухудшает качество
   - Слишком малый learning_rate (<0.05) требует больше деревьев для сходимости
   
3. СРАВНЕНИЕ АЛГОРИТМОВ:
   - Gradient Boosting обычно показывает лучшее качество при правильной настройке
   - Random Forest быстрее обучается и менее чувствителен к гиперпараметрам
   - Random Forest легче параллелится (независимые деревья)
   - Gradient Boosting требует последовательного обучения деревьев
   
4. ПРАКТИЧЕСКИЕ РЕКОМЕНДАЦИИ:
   - Для быстрого прототипирования: Random Forest с дефолтными параметрами
   - Для максимального качества: Gradient Boosting с подбором гиперпараметров
   - Для больших данных: Random Forest (лучше масштабируется)
   - Для интерпретируемости: Random Forest (более стабильные предсказания)
""")

In [ ]:

gbr_grid = {
    "n_estimators": [50, 100, 200],
    "max_features": ["sqrt", None],
    "max_depth": [3, None],
    "learning_rate": [0.05, 0.1, 0.2],
}

gbr_results = run_grid(GradientBoostingRegressor, gbr_grid, "GradientBoosting")
gbr_results.sort_values("rmse").head()
